In [1]:
from pathlib import Path
from skimage.color import separate_stains, hed_from_rgb
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import tifffile
import openslide
import gc
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
import plotly.express as px
from utils_stardist import get_gpu_details, list_images, extract_nuclei_stack, get_stardist_model, maximum_intensity_projection, save_rois, simulate_cytoplasm_chunked_3d, simulate_cell_chunked_3d, simulate_cytoplasm, simulate_cell, segment_nuclei, remove_labels_touching_roi_edge

get_gpu_details()

model = StarDist2D.from_pretrained('2D_versatile_fluo')

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/SMA_staining/SMA staining/")

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path, format="ndpi")

images

['..\\raw_data\\SMA_staining\\SMA staining\\1361 436 MD SMA - 2016-04-15 07.36.02.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1361 436+912 MI SMA - 2016-04-15 07.45.40.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1362 436 MD SMA - 2016-04-15 07.50.57.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1362 436+912 MI SMA - 2016-04-15 08.01.21.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1363 436 MD SMA - 2016-04-15 08.12.14.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1363 436+907 MI SMA - 2016-04-15 08.17.47.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1364 436 MD SMA - 2016-04-15 08.29.51.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1364 436+907 MI SMA - 2016-04-15 09.02.39.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1368 436+907 MI SMA - 2016-04-15 09.20.09.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1368 436+912 MD SMA - 2016-04-15 09.08.14.ndpi',
 '..\\raw_data\\SMA_staining\\SMA staining\\1369 436+907 MI SMA - 2016-04-15 09.33.49.

In [3]:
slide = openslide.OpenSlide(images[0])

# Get level 0 dimensions (highest resolution)
# width, height = slide.dimensions

# Optionally, you can select a lower resolution level for faster loading
level = 1  # for example
width, height = slide.level_dimensions[level]

# Read region from level 0 (top-left corner, full size)
# The region is returned as a PIL Image
region = slide.read_region(location=(0, 0), level=1, size=(width, height))

# Remove slide variable
del slide
gc.collect()

# Convert to RGB (removes alpha channel) and then to NumPy array
region_rgb = region.convert("RGB")
image_np = np.array(region_rgb)

# Perform color deconvolution
deconvolved = separate_stains(image_np, hed_from_rgb)
nuclei_img = deconvolved[:,:,0]
dab_img = deconvolved[:,:,2]

In [ ]:
prob_thresh = None
nms_thresh = None

labels, _ = model.predict_instances(normalize(deconvolved[:,:,0]),
                                    n_tiles = (3,3), 
                                    prob_thresh=prob_thresh, 
                                    nms_thresh=nms_thresh)

In [15]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(image_np)
viewer.add_image(nuclei_img, name="Nuclei")
viewer.add_image(dab_img, name="DAB")
viewer.add_labels(labels, name="nuclei_labels")

<Labels layer 'nuclei_labels' at 0x18bc127bfd0>

In [ ]:
#TODO: Batch predict and save nuclei labels, modify read image function for ndpi and color deconv, batch predict nuclei, train obj classifier